**STEP 1: DATA COLLECTION AND EXPLORATION**


_ALL THE IMPORTS_


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import GridSearchCV
import pickle

_LOAD THE DATASET_


In [3]:
# Loading the dataset
df = pd.read_csv("../datasets/grocery_chain_data.csv")

df

,customer_id,store_name,transaction_date,aisle,product_name,quantity,unit_price,total_amount,discount_amount,final_amount,loyalty_points
0,2824,GreenGrocer Plaza,2023-08-26,Produce,Pasta,2.0,7.46,14.92,0.00,14.92,377
1,5506,ValuePlus Market,2024-02-13,Dairy,Cheese,1.0,1.85,1.85,3.41,-1.56,111
2,4657,ValuePlus Market,2023-11-23,Bakery,Onions,4.0,7.38,29.52,4.04,25.48,301
3,2679,SuperSave Central,2025-01-13,Snacks & Candy,Cereal,3.0,5.50,16.50,1.37,15.13,490
4,9935,GreenGrocer Plaza,2023-10-13,Canned Goods,Orange Juice,5.0,8.66,43.30,1.50,41.80,22
...,...,...,...,...,...,...,...,...,...,...,...
1975,1699,Corner Grocery,2024-07-30,Frozen Foods,Salmon,3.0,9.79,29.37,4.41,24.96,274
1976,1829,City Fresh Store,2023-12-15,Personal Care,Potatoes,4.0,22.96,91.84,9.18,82.66,429
1977,8096,MegaMart Westside,2025-05-06,Dairy,Potatoes,3.0,16.44,49.32,7.40,41.92,315
1978,7471,QuickStop Market,2024-08-26,Frozen Foods,Rice,5.0,15.88,79.40,15.88,63.52,160


_INITIAL EXPLORATION_


In [6]:
# Shape of the dataset
df.shape

(1980, 11)

In [7]:
# Data types of the dataset
df.dtypes

customer_id           int64
store_name           object
transaction_date     object
aisle                object
product_name         object
quantity            float64
unit_price          float64
total_amount        float64
discount_amount     float64
final_amount        float64
loyalty_points        int64
dtype: object

In [9]:
# Summary statistics
df.describe()

,customer_id,quantity,unit_price,total_amount,discount_amount,final_amount,loyalty_points
count,1980.000000,1980.000000,1980.000000,1980.000000,1980.000000,1980.000000,1980.000000
mean,5542.958081,2.968182,15.488045,45.902576,4.469591,41.432985,255.147980
std,2575.771856,1.419028,8.400823,35.018599,4.962001,32.593328,146.009333
min,1006.000000,1.000000,0.990000,1.010000,0.000000,-3.430000,0.000000
25%,3271.500000,2.000000,8.240000,18.000000,1.240000,15.800000,128.000000
50%,5582.500000,3.000000,15.190000,37.130000,3.045000,32.820000,265.500000
75%,7791.750000,4.000000,22.862500,67.930000,5.402500,60.800000,378.000000
max,9998.000000,5.000000,29.980000,149.900000,29.940000,147.910000,500.000000


In [10]:
# Finding missing values
df.isna().sum()

customer_id          0
store_name          25
transaction_date     0
aisle                0
product_name         0
quantity             0
unit_price           0
total_amount         0
discount_amount      0
final_amount         0
loyalty_points       0
dtype: int64

In [11]:
# Checking for duplicate data
df.duplicated().sum()

np.int64(0)

In [ ]:
# Checking for negative values in columns like "quantity", "unit_price", "total_amount", "discount_amount", "final_amount", "loyalty_points" as they should be either 0 or greater
num_cols = [
    "quantity",
    "unit_price",
    "total_amount",
    "discount_amount",
    "final_amount",
    "loyalty_points",
]

# Using for loop for checking negative values for each numerical cols above
for col in num_cols:
    print(f"{col}: {(df[col] < 0).sum()} negative values")

quantity: 0 negative values
unit_price: 0 negative values
total_amount: 0 negative values
discount_amount: 0 negative values
final_amount: 13 negative values
loyalty_points: 0 negative values


**STEP 2: DATA CLEANING AND TRANSFORMATION**


_HANDLING MISSING VALUES_


In [ ]:
# Removing the row with missing store name
df = df.dropna(subset=["store_name"])

In [21]:
# Shape of the dataset after removing row with missing store name
df.shape

(1955, 11)

In [25]:
# Converting transaction date from "object" type to "datatime"
df["transaction_date"] = pd.to_datetime(df["transaction_date"])

In [32]:
# Extract features like year, month, day, and wwekday from "transaction_date"
df["transaction_year"] = df["transaction_date"].dt.year  # transaction_year
df["transaction_month"] = df["transaction_date"].dt.month  # transaction_month
df["transaction_day"] = df["transaction_date"].dt.day  # transaction_day
df["transaction_weekday"] = df["transaction_date"].dt.weekday  # transaction_weekday

df

,customer_id,store_name,transaction_date,aisle,product_name,quantity,unit_price,total_amount,discount_amount,final_amount,loyalty_points,transaction_year,transaction_month,transaction_day,transaction_weekday
0,2824,GreenGrocer Plaza,2023-08-26,Produce,Pasta,2.0,7.46,14.92,0.00,14.92,377,2023,8,26,5
1,5506,ValuePlus Market,2024-02-13,Dairy,Cheese,1.0,1.85,1.85,3.41,-1.56,111,2024,2,13,1
2,4657,ValuePlus Market,2023-11-23,Bakery,Onions,4.0,7.38,29.52,4.04,25.48,301,2023,11,23,3
3,2679,SuperSave Central,2025-01-13,Snacks & Candy,Cereal,3.0,5.50,16.50,1.37,15.13,490,2025,1,13,0
4,9935,GreenGrocer Plaza,2023-10-13,Canned Goods,Orange Juice,5.0,8.66,43.30,1.50,41.80,22,2023,10,13,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1975,1699,Corner Grocery,2024-07-30,Frozen Foods,Salmon,3.0,9.79,29.37,4.41,24.96,274,2024,7,30,1
1976,1829,City Fresh Store,2023-12-15,Personal Care,Potatoes,4.0,22.96,91.84,9.18,82.66,429,2023,12,15,4
1977,8096,MegaMart Westside,2025-05-06,Dairy,Potatoes,3.0,16.44,49.32,7.40,41.92,315,2025,5,6,1
1978,7471,QuickStop Market,2024-08-26,Frozen Foods,Rice,5.0,15.88,79.40,15.88,63.52,160,2024,8,26,0


In [33]:
# Handling negative values in "final_amount" column by replacing it with 0 assuming they are due to errors
df["final_amount"] = df["final_amount"].apply(lambda x: max(x, 0))

df

,customer_id,store_name,transaction_date,aisle,product_name,quantity,unit_price,total_amount,discount_amount,final_amount,loyalty_points,transaction_year,transaction_month,transaction_day,transaction_weekday
0,2824,GreenGrocer Plaza,2023-08-26,Produce,Pasta,2.0,7.46,14.92,0.00,14.92,377,2023,8,26,5
1,5506,ValuePlus Market,2024-02-13,Dairy,Cheese,1.0,1.85,1.85,3.41,0.00,111,2024,2,13,1
2,4657,ValuePlus Market,2023-11-23,Bakery,Onions,4.0,7.38,29.52,4.04,25.48,301,2023,11,23,3
3,2679,SuperSave Central,2025-01-13,Snacks & Candy,Cereal,3.0,5.50,16.50,1.37,15.13,490,2025,1,13,0
4,9935,GreenGrocer Plaza,2023-10-13,Canned Goods,Orange Juice,5.0,8.66,43.30,1.50,41.80,22,2023,10,13,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1975,1699,Corner Grocery,2024-07-30,Frozen Foods,Salmon,3.0,9.79,29.37,4.41,24.96,274,2024,7,30,1
1976,1829,City Fresh Store,2023-12-15,Personal Care,Potatoes,4.0,22.96,91.84,9.18,82.66,429,2023,12,15,4
1977,8096,MegaMart Westside,2025-05-06,Dairy,Potatoes,3.0,16.44,49.32,7.40,41.92,315,2025,5,6,1
1978,7471,QuickStop Market,2024-08-26,Frozen Foods,Rice,5.0,15.88,79.40,15.88,63.52,160,2024,8,26,0


In [39]:
df["product_name"].unique()

array(['Pasta', 'Cheese', 'Onions', 'Cereal', 'Orange Juice',
       'Ground Beef', 'Apples', 'Tomatoes', 'Salmon', 'Rice', 'Bananas',
       'Milk', 'Eggs', 'Bread', 'Yogurt', 'Carrots', 'Potatoes',
       'Chicken Breast'], dtype=object)

In [ ]:
# There are some inconsistencies like "Potatoes" appearing in multiple aisles like in "Personal Care", "Dairy" like in multiple aisles.
# So, creating a product category mapping to standardize
product_category_map = {
    "Pasta": "Dry Goods",
    "Cheese": "Dairy",
    "Onions": "Produce",
    "Cereal": "Breakfast",
    "Orange Juice": "Beverages",
    "Ground Beef": "Meat",
    "Apples": "Produce",
    "Tomatoes": "Produce",
    "Salmon": "Seafood",
    "Rice": "Grains",
    "Bananas": "Produce",
    "Milk": "Dairy",
    "Eggs": "Dairy",
    "Bread": "Bakery",
    "Yogurt": "Dairy",
    "Carrots": "Produce",
    "Potatoes": "Produce",
    "Chicken Breast": "Meat",
}

df["product_category"] = df["product_name"].map(product_category_map)

df


,customer_id,store_name,transaction_date,aisle,product_name,quantity,unit_price,total_amount,discount_amount,final_amount,loyalty_points,transaction_year,transaction_month,transaction_day,transaction_weekday,product_category
0,2824,GreenGrocer Plaza,2023-08-26,Produce,Pasta,2.0,7.46,14.92,0.00,14.92,377,2023,8,26,5,Dry Goods
1,5506,ValuePlus Market,2024-02-13,Dairy,Cheese,1.0,1.85,1.85,3.41,0.00,111,2024,2,13,1,Dairy
2,4657,ValuePlus Market,2023-11-23,Bakery,Onions,4.0,7.38,29.52,4.04,25.48,301,2023,11,23,3,Produce
3,2679,SuperSave Central,2025-01-13,Snacks & Candy,Cereal,3.0,5.50,16.50,1.37,15.13,490,2025,1,13,0,Breakfast
4,9935,GreenGrocer Plaza,2023-10-13,Canned Goods,Orange Juice,5.0,8.66,43.30,1.50,41.80,22,2023,10,13,4,Beverages
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1975,1699,Corner Grocery,2024-07-30,Frozen Foods,Salmon,3.0,9.79,29.37,4.41,24.96,274,2024,7,30,1,Seafood
1976,1829,City Fresh Store,2023-12-15,Personal Care,Potatoes,4.0,22.96,91.84,9.18,82.66,429,2023,12,15,4,Produce
1977,8096,MegaMart Westside,2025-05-06,Dairy,Potatoes,3.0,16.44,49.32,7.40,41.92,315,2025,5,6,1,Produce
1978,7471,QuickStop Market,2024-08-26,Frozen Foods,Rice,5.0,15.88,79.40,15.88,63.52,160,2024,8,26,0,Grains


In [42]:
# Calculate discount percentage
df["discount_percent"] = (df["discount_amount"] / df["total_amount"]).fillna(0)

df

,customer_id,store_name,transaction_date,aisle,product_name,quantity,unit_price,total_amount,discount_amount,final_amount,loyalty_points,transaction_year,transaction_month,transaction_day,transaction_weekday,product_category,discount_percent
0,2824,GreenGrocer Plaza,2023-08-26,Produce,Pasta,2.0,7.46,14.92,0.00,14.92,377,2023,8,26,5,Dry Goods,0.000000
1,5506,ValuePlus Market,2024-02-13,Dairy,Cheese,1.0,1.85,1.85,3.41,0.00,111,2024,2,13,1,Dairy,1.843243
2,4657,ValuePlus Market,2023-11-23,Bakery,Onions,4.0,7.38,29.52,4.04,25.48,301,2023,11,23,3,Produce,0.136856
3,2679,SuperSave Central,2025-01-13,Snacks & Candy,Cereal,3.0,5.50,16.50,1.37,15.13,490,2025,1,13,0,Breakfast,0.083030
4,9935,GreenGrocer Plaza,2023-10-13,Canned Goods,Orange Juice,5.0,8.66,43.30,1.50,41.80,22,2023,10,13,4,Beverages,0.034642
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1975,1699,Corner Grocery,2024-07-30,Frozen Foods,Salmon,3.0,9.79,29.37,4.41,24.96,274,2024,7,30,1,Seafood,0.150153
1976,1829,City Fresh Store,2023-12-15,Personal Care,Potatoes,4.0,22.96,91.84,9.18,82.66,429,2023,12,15,4,Produce,0.099956
1977,8096,MegaMart Westside,2025-05-06,Dairy,Potatoes,3.0,16.44,49.32,7.40,41.92,315,2025,5,6,1,Produce,0.150041
1978,7471,QuickStop Market,2024-08-26,Frozen Foods,Rice,5.0,15.88,79.40,15.88,63.52,160,2024,8,26,0,Grains,0.200000


In [ ]:
# Calculate price per unit (some items are sold in qunatities > 1)
# df["price"]

In [ ]:
# Create a loyalty tier based on points
def loyalty_tier(points):
    if points < 100:
        return "Bronze"
    elif points < 300:
        return "Silver"
    else:
        return "Gold"


df["loyalty_tier"] = df["loyalty_points"].apply(loyalty_tier)

df

,customer_id,store_name,transaction_date,aisle,product_name,quantity,unit_price,total_amount,discount_amount,final_amount,loyalty_points,transaction_year,transaction_month,transaction_day,transaction_weekday,product_category,discount_percent,loyalty_tier
0,2824,GreenGrocer Plaza,2023-08-26,Produce,Pasta,2.0,7.46,14.92,0.00,14.92,377,2023,8,26,5,Dry Goods,0.000000,Gold
1,5506,ValuePlus Market,2024-02-13,Dairy,Cheese,1.0,1.85,1.85,3.41,0.00,111,2024,2,13,1,Dairy,1.843243,Silver
2,4657,ValuePlus Market,2023-11-23,Bakery,Onions,4.0,7.38,29.52,4.04,25.48,301,2023,11,23,3,Produce,0.136856,Gold
3,2679,SuperSave Central,2025-01-13,Snacks & Candy,Cereal,3.0,5.50,16.50,1.37,15.13,490,2025,1,13,0,Breakfast,0.083030,Gold
4,9935,GreenGrocer Plaza,2023-10-13,Canned Goods,Orange Juice,5.0,8.66,43.30,1.50,41.80,22,2023,10,13,4,Beverages,0.034642,Bronze
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1975,1699,Corner Grocery,2024-07-30,Frozen Foods,Salmon,3.0,9.79,29.37,4.41,24.96,274,2024,7,30,1,Seafood,0.150153,Silver
1976,1829,City Fresh Store,2023-12-15,Personal Care,Potatoes,4.0,22.96,91.84,9.18,82.66,429,2023,12,15,4,Produce,0.099956,Gold
1977,8096,MegaMart Westside,2025-05-06,Dairy,Potatoes,3.0,16.44,49.32,7.40,41.92,315,2025,5,6,1,Produce,0.150041,Gold
1978,7471,QuickStop Market,2024-08-26,Frozen Foods,Rice,5.0,15.88,79.40,15.88,63.52,160,2024,8,26,0,Grains,0.200000,Silver


**STEP 3: EXPLORATORY DATA ANALYSIS (EDA)**
